In [ ]:
import numpy as np
import requests
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from math import floor
from termcolor import colored as cl

plt.style.use("fivethirtyeight")
plt.rcParams["figure.figsize"] = (20, 10)